In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")


assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."
assert TAVILY_API_KEY, "TAVILY_API_KEY is missing. Check your .env file."

print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")
print("Tavily Key loaded:", TAVILY_API_KEY[:6] + "...")

# Initialize LLMs
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY
)
openai_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# Prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="Give me a creative name for a company that makes {product}?",
)

# Example: Use Gemini
chain = prompt | gemini_llm
response = chain.invoke({"product": "smart home devices"})
print("Gemini response:", response)

# Example: Use OpenAI
chain = prompt | openai_llm
response = chain.invoke({"product": "smart home devices"})
print("OpenAI response:", response)

Key loaded: AIzaSy...
OpenAI Key loaded: sk-pro...
Tavily Key loaded: tvly-d...
Gemini response: content='Here are some creative name ideas for a smart home device company, categorized by their evoked feeling or style:\n\n## Modern & Tech-Forward\n\n1.  **AuraHome:** Evokes a feeling of an invisible, intelligent presence enhancing the home.\n2.  **Synapse Living:** Suggests seamless connections and intelligent control, like the brain\'s synapses.\n3.  **Veridian Tech:** "Verdant" implies lush growth and vitality; "Veridian" sounds modern and clean, hinting at a natural, effortless integration of tech.\n4.  **Kinetics Dwell:** Focuses on dynamic, responsive systems that adapt to your movements and needs.\n5.  **Flux Systems:** Implies constant adaptation, flow, and evolution of the home environment.\n6.  **Nexus Dwellings:** Positions the company as the central connecting point for all home intelligence.\n7.  **Lumenos:** A blend of "lumen" (light, brilliance) and "os" (operating system

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Prompt + LLM
prompt = PromptTemplate(
    input_variables=["product"],
    template="List 3 short tagline ideas for a {product}, one per line.",
)

# ---------- WITHOUT output parser ----------
chain = prompt | openai_llm
raw_result = chain.invoke({"product": "smart home devices"})
print("WITHOUT parser -> type:", type(raw_result))
print("WITHOUT parser -> value:", raw_result, "\n")

# ---------- WITH output parser (StrOutputParser) ----------
# StrOutputParser turns AIMessage -> plain string
parsed_chain = chain | StrOutputParser()
parsed_result = parsed_chain.invoke({"product": "smart home devices"})
print("WITH parser -> type:", type(parsed_result))
print("WITH parser -> value:", parsed_result)

WITHOUT parser -> type: <class 'langchain_core.messages.ai.AIMessage'>
WITHOUT parser -> value: content='1. "Simplify your life with smart home technology."\n2. "Upgrade to a smarter, more efficient home."\n3. "Experience the future of living with smart home devices."' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 24, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CqdzPb6v2QoXbUjDB9m8kd3KhQjyN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b554f-2c07-7053-a366-5894d7b5aa80-0' usage_metadata={'input_tokens': 24, 'output_tokens': 37, 'total_tokens': 61, 'input_token_details': {'audio': 0, 'cache_read'

In [7]:
from langchain_core.output_parsers import JsonOutputParser

# ---------- WITH JSON output parser (JsonOutputParser) ----------
# Using PromptTemplate.from_template (no need to list input_variables)
json_prompt = PromptTemplate.from_template(
    "Return a JSON object with a single key 'taglines' that is a list of 3 short taglines "
    "for the product: {product}."
)

# See what the final prompt text looks like
formatted_prompt = json_prompt.format(product="smart home devices")
print("Formatted prompt text:\n", formatted_prompt, "\n")

json_chain = json_prompt | openai_llm
json_parsed_chain = json_chain | JsonOutputParser()

json_result = json_parsed_chain.invoke({"product": "smart home devices"})
print("JSON parser -> type:", type(json_result))
print("JSON parser -> value:", json_result)   # {"taglines": [...]}

Formatted prompt text:
 Return a JSON object with a single key 'taglines' that is a list of 3 short taglines for the product: smart home devices. 

JSON parser -> type: <class 'dict'>
JSON parser -> value: {'taglines': ['Make your home smarter', 'Connect, automate, simplify', 'Upgrade to a smarter lifestyle']}


In [10]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

# Define the output shape
class Celebrity(BaseModel):
    name: str = Field(description="Celebrity full name")
    age: int = Field(description="Approximate age in years")
    country: str = Field(description="Main country associated with the celebrity")

# Parser for that shape
celeb_parser = PydanticOutputParser(pydantic_object=Celebrity)

# Prompt that tells the model to follow this shape
celeb_prompt = PromptTemplate.from_template(
    "{format_instructions}\n\nCelebrity: {celebrity_name}"
)

format_instructions = celeb_parser.get_format_instructions()

# Chain: prompt -> LLM -> parser
celeb_chain = celeb_prompt | openai_llm | celeb_parser

# Call it
result = celeb_chain.invoke(
    {
        "celebrity_name": "Tom Cruise",
        "format_instructions": format_instructions,
    }
)

print(result.dict())   # simple JSON: {"name": ..., "age": ..., "country": ...}

{'name': 'Tom Cruise', 'age': 59, 'country': 'United States'}


/var/folders/q1/q82cy9ks5kj94l08331njly40000gn/T/ipykernel_82962/1252347661.py:32: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  print(result.dict())   # simple JSON: {"name": ..., "age": ..., "country": ...}
